# Non-relational database using neo4j
In this notebook we will learn how to create a non-relational database using neo4j

Head over to https://neo4j.com/sandbox/ , create an account, create an instance and get the credentials to the instance.

Prerequisites:
* pip install osmnx neo4j pandas networkx psycopg2-binary tqdm matplotlib

### Step 1: Install the required libraries

You can install the required libraries via pip if you haven't already:

`!pip install osmnx neo4j pandas networkx tqdm matplotlib`

### Step 2: Import necessary libraries

In [3]:
import networkx as nx # NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.
from neo4j import GraphDatabase # Neo4j is a graph database management system developed by Neo4j, Inc.
import pandas as pd # pandas is a software library written for the Python programming language for data manipulation and analysis.
from tqdm import tqdm # tqdm is a Python library that allows you to output a smart progress bar by wrapping around any iterable.
import osmnx as ox # OSMnx is a Python package that lets you download spatial geometries and construct, project, and visualize street networks from OpenStreetMap's APIs.
import matplotlib.pyplot as plt # Matplotlib is a plotting library for the Python programming language and its numerical mathematics extension NumPy.

#### Google Colab Users:

In [ ]:
# Run this block only if you want to use Google Colab
# Install the google-colab package by !pip install google-colab
# Get access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Then you need to upload the data to your Google Drive and change the path to the data by right clicking on the file and selecting "Copy path" and pasting it

# Demo 

In this Demo we will create a graph-based database for the Washington DC Metro Area bikesharing trip data. [Link](https://capitalbikeshare.com/system-data)

Download the dataset included in the D2L assignment named `Tripdata.Zip` and extract it in the same directory as the `Part_2_Non_Relational_database.ipynb` file.

The Goal:

Insert the data to the Neo4j database and retrive information using Cypher queries.

### Data Preprocessing

In [5]:
# Import data from - 202001-Washington DC Metro Area-bikeshare-tripdata.csv
data = pd.read_csv("202001-Washington DC Metro Area-bikeshare-tripdata.csv")
# Print the first 5 rows of the data
data.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,506,2020-01-01 00:00:59,2020-01-01 00:09:26,31623,Columbus Circle / Union Station,31658,Rosedale Rec Center,W22189,Member
1,1050,2020-01-01 00:01:16,2020-01-01 00:18:47,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31606,Potomac & Pennsylvania Ave SE,W21067,Member
2,1687,2020-01-01 00:02:42,2020-01-01 00:30:50,31218,L'Enfant Plaza / 7th & C St SW,31271,Constitution Ave & 2nd St NW/DOL,W24359,Member
3,182,2020-01-01 00:02:44,2020-01-01 00:05:47,31627,3rd & M St NE,31508,Gallaudet / 8th St & Florida Ave NE,W23334,Member
4,1002,2020-01-01 00:06:12,2020-01-01 00:22:54,31247,Jefferson Dr & 14th St SW,31274,10th & G St NW,W23441,Member


In [6]:
# Print the data types of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196969 entries, 0 to 196968
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Duration              196969 non-null  int64 
 1   Start date            196969 non-null  object
 2   End date              196969 non-null  object
 3   Start station number  196969 non-null  int64 
 4   Start station         196969 non-null  object
 5   End station number    196969 non-null  int64 
 6   End station           196969 non-null  object
 7   Bike number           196969 non-null  object
 8   Member type           196969 non-null  object
dtypes: int64(3), object(6)
memory usage: 13.5+ MB


In [7]:
# Let's sort the data by 'start_time' in ascending order and select the first 2 days of data
data_sorted = data.sort_values('Start date')
data_2days = data_sorted[data_sorted['Start date'] < '2020-01-02']


In [8]:
# Unique values in the 'start_station_id' column
start_stations = data_2days['Start station number'].unique()
print(f"Number of unique start stations: {len(start_stations)}")
# Unique values in the 'end_station_id' column
end_stations = data_2days['End station number'].unique()
print(f"Number of unique end stations: {len(end_stations)}")
# Unique values in the 'Bike number' column
bikes = data_2days['Bike number'].unique()
print(f"Number of unique bikes: {len(bikes)}")
# Unique values in the 'Member type' column
member_types = data_2days['Member type'].unique()
print(f"Types of members: {member_types}")


Number of unique start stations: 431
Number of unique end stations: 435
Number of unique bikes: 1846
Types of members: ['Member' 'Casual']


Notice that we only have the location and not the coordinates. Here is how we can Geocode a location in Python:

In [9]:
# Here is how you can geocode the Start station and End station addresses using the osmnx library:
import osmnx as ox

# Initialize the geocoder
address = "3rd & M St NE, Washington DC"

location = ox.geocode(address)

# Get latitude and longitude
if location:
    print(f"Address: {address}, Latitude: {location[0]}, Longitude: {location[1]}")
else:
    print("Location not found")


Address: 3rd & M St NE, Washington DC, Latitude: 38.9056467, Longitude: -77.0074027


In this exercise we don't Geocode all the dataframe locations but it is possible to do so using OSMnx library

#### ER Diagram for Bikesharing Data

##### Nodes

##### Station
- **Properties:**
  - `number` (integer): The station number.
  - `name` (string): The name or description of the station.

##### Bike
- **Properties:**
  - `number` (string): The bike number.

##### Member
- **Properties:**
  - `type` (string): The type of member (e.g., "Member").

##### Trip
- **Properties:**
  - `duration` (integer): Duration of the trip in seconds.
  - `startDate` (datetime): The start date and time of the trip.
  - `endDate` (datetime): The end date and time of the trip.

#### Relationships (Edges)

##### STARTED_AT
- **From:** `Trip`
- **To:** `Station`
- **Description:** Represents the starting station of the trip.

##### ENDED_AT
- **From:** `Trip`
- **To:** `Station`
- **Description:** Represents the ending station of the trip.

##### USED
- **From:** `Trip`
- **To:** `Bike`
- **Description:** Indicates which bike was used during the trip.

##### SUBSCRIBED_BY
- **From:** `Trip`
- **To:** `Member`
- **Description:** Indicates which member (if any) subscribed to the trip.

#### Diagram Visualization
Refer to the slides.

In a visual ER diagram:

- **Nodes** will be represented as entities with their attributes shown inside or near them.
- **Relationships** will be depicted as arrows connecting nodes, with labels indicating the type of relationship.

#### Textual Representation



In [ ]:
# Here’s a rough textual representation of the diagram:
"""
[Trip] -[STARTED_AT]-> [Station]
   |
   v
[Trip] -[ENDED_AT]-> [Station]
   |
   v
[Trip] -[USED]-> [Bike]
   |
   v
[Trip] -[SUBSCRIBED_BY]-> [Member]

"""

#### Neo4j Helper Class:

Fill the #*# spaces inside the following block to add the data to the database. Each empty space holds `10 points`. Total: `30 Points`

In [10]:
from neo4j import GraphDatabase
from tqdm import tqdm  # For progress bar
import pandas as pd  # Assuming you have the data in a DataFrame

# Neo4j connection details - Get these from your Neo4j sandbox instance
uri = "neo4j+s://19cab18e.databases.neo4j.io"  # Adjust if necessary
username = "neo4j"  # Default Neo4j username
password = "JKKbI4RiaAUBfp1Oz8uztlpo6B81KoD9IrGukvjoaFI"  # Replace with your password

# Batch size for processing nodes and edges
BATCH_SIZE = 100  # Adjust batch size as needed based on the size of the dataset

# Convert the data into a list of dictionaries for nodes and edges
nodes = []
edges = []

# Add stations
stations = set(data_2days['Start station']).union(set(data_2days['End station']))
for station in stations:
    nodes.append({"name": station, "type": "Station"})

# Add bikes
bikes = set(data_2days['Bike number'])
for bike in bikes:
    nodes.append({"name": bike, "type": "Bike"})

# Add members
members = set(data_2days['Member type'])
for member in members:
    nodes.append({"name": member, "type": "Member"})

# Add trips and edges
for id, row in data_2days.iterrows():
    # consider using a row index as the trip_id
    trip_id = id
    
    nodes.append({"id": trip_id, "type": "Trip", "duration": row['Duration'], "startDate": row['Start date'], "endDate": row['End date']})

    edges.append({
        "source": trip_id,
        "target": row['Start station'],
        "relationship": "STARTED_AT"
    })
    edges.append({
        "source": trip_id,
        "target": row['End station'],
        "relationship": "ENDED_AT"
    })
    edges.append({
        "source": trip_id,
        "target": row['Bike number'],
        "relationship": "USED"
    })
    edges.append({
        "source": trip_id,
        "target": row['Member type'],
        "relationship": "SUBSCRIBED_BY"
    })

# Neo4j Driver class
class Neo4jGraph:
    # Initialize the Neo4j driver
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    # Close the Neo4j driver
    def close(self):
        self.driver.close()

    # Insert the graph data into Neo4j
    def insert_graph(self, nodes, edges):
        with self.driver.session() as session:
            # Insert nodes in batches
            print("Inserting nodes...")
            for i in tqdm(range(0, len(nodes), BATCH_SIZE)):
                batch = nodes[i:i + BATCH_SIZE]
                session.execute_write(self._create_nodes_batch, batch)
            print("Nodes inserted successfully!")
            print("Inserting edges...")
            # Insert edges in batches
            for i in tqdm(range(0, len(edges), BATCH_SIZE)):
                batch = edges[i:i + BATCH_SIZE]
                session.execute_write(self._create_edges_batch, batch)
            print("Edges inserted successfully!")

    # Create nodes in Neo4j
    @staticmethod
    def _create_nodes_batch(tx, batch):
        # for row in batch:
        #     query = f"""
        #     CREATE (n:{row['type']})
        #     SET n = $props
        #     """
        #     tx.run(query, props=row)
        query = """
        UNWIND $batch AS row
        CALL apoc.create.node([row.type], row) YIELD node
        RETURN node
        """
        tx.run(query, batch=batch)

    # Create edges in Neo4j
    @staticmethod
    def _create_edges_batch(tx, batch):
        # for row in batch:
        #     query = f"""
        #     MATCH (source), (target)
        #     WHERE source.id = $source AND target.name = $target
        #     CREATE (source)-[:{row['relationship']}]->(target)
        #     """
        #     tx.run(query, source=row['source'], target=row['target'])
        query = """
        UNWIND $batch AS row
        MATCH (source {id: row.source}), (target {name: row.target})
        CALL apoc.create.relationship(source, row.relationship, {}, target) YIELD rel
        RETURN rel
        """
        tx.run(query, batch=batch)

    # Execute a Cypher query in Neo4j
    def query(self, query, parameters=None, db=None):
        # Ensure the driver is initialized
        assert self.driver is not None, "Driver not initialized!"
        session = None  # Initialize the session
        response = None  # Initialize the response
        try:
            # Connect to the database (if specified) or the default database
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            # Execute the query
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            # Close the session
            if session is not None:
                session.close()
        return response

# Insert the graph data into Neo4j
neo4j_graph = Neo4jGraph(uri, username, password)

In [11]:
# number of nodes and edges
num_nodes = len(nodes)
num_edges = len(edges)
# print the number of nodes and edges
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

Number of nodes: 6447
Number of edges: 16540


In [12]:
# Clear the graph database
neo4j_graph.query("MATCH (n) DETACH DELETE n")

[]

In [13]:
# get the last 5 edges
edges[-5:]

[{'source': 4133, 'target': 'Member', 'relationship': 'SUBSCRIBED_BY'},
 {'source': 4134,
  'target': 'Columbus Circle / Union Station',
  'relationship': 'STARTED_AT'},
 {'source': 4134,
  'target': 'Eastern Market / 7th & North Carolina Ave SE',
  'relationship': 'ENDED_AT'},
 {'source': 4134, 'target': 'W21721', 'relationship': 'USED'},
 {'source': 4134, 'target': 'Member', 'relationship': 'SUBSCRIBED_BY'}]

In [14]:
# Insert the graph data into Neo4j
neo4j_graph.insert_graph(nodes, edges)
print("Data successfully inserted in batches!")

Inserting nodes...


100%|██████████| 65/65 [00:13<00:00,  4.98it/s]


Nodes inserted successfully!
Inserting edges...


100%|██████████| 166/166 [01:41<00:00,  1.63it/s]

Edges inserted successfully!
Data successfully inserted in batches!


### Questions: Complete the queries:

Answer the follwing questions by wrting queries. Each question holds `10 points`.

In [80]:
# Question 1: What are the top 10 most popular start stations for bike trips?
query = """
MATCH (s:Station)<-[:STARTED_AT]-()
RETURN s.name AS StartStation, count(*) AS Trips
ORDER BY Trips DESC
LIMIT 10
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 1:")
print(result_df)

# Question 2: What are the top 10 most popular end stations for bike trips?
query = """
MATCH (s:Station)<-[:ENDED_AT]-()
RETURN s.name AS EndStation, count(*) AS Trips  
ORDER BY Trips DESC
LIMIT 10
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 2:")
print(result_df)

# Question 3: What is the average duration of bike trips by member type?
query = """
MATCH (m:Member)<-[:SUBSCRIBED_BY]-(t:Trip)
RETURN m.type AS MemberType, avg(t.duration) AS AvgDuration
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 3:")
print(result_df)

# Question 4: How many unique bikes were used in the dataset?
query = """
MATCH (b:Bike)
RETURN count(DISTINCT b.name) AS UniqueBikes
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 4:")
print(result_df)

# Question 5: How many trips were taken by each member type?
query = """
MATCH (m:Member)<-[:SUBSCRIBED_BY]-(t:Trip)
RETURN m.type AS MemberType, count(*) AS Trips
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 5:")
print(result_df)

# Question 6: What are the top 10 most used bikes for the trips?
query_string = '''
MATCH (b:Bike)<-[:USED]-()
RETURN b.name AS Bike, count(*) AS Trips
ORDER BY Trips DESC
LIMIT 10
'''

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query_string)])
# Print the result
print("Answer to Question 6:")
print(result_df)



C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database=db) if db is not None else self.driver.session()
C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database=db) if db is not None else self.driver.session()
C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database=db) if db is not None else self.driver.session()
C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Futur

Answer to Question 1:
                                        StartStation  Trips
0                                   Lincoln Memorial     76
1                          Jefferson Dr & 14th St SW     68
2                             4th St & Madison Dr NW     67
3                        New Hampshire Ave & T St NW     57
4                                     15th & P St NW     56
5        Henry Bacon Dr & Lincoln Memorial Circle NW     53
6  Smithsonian-National Mall / Jefferson Dr & 12t...     53
7                    Columbus Circle / Union Station     49
8                      15th St & Constitution Ave NW     46
9                                14th & Irving St NW     46
Answer to Question 2:
                                          EndStation  Trips
0                                   Lincoln Memorial     76
1                             4th St & Madison Dr NW     66
2                          Jefferson Dr & 14th St SW     66
3  Smithsonian-National Mall / Jefferson Dr & 12t...    

C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database=db) if db is not None else self.driver.session()
C:\Users\reals\AppData\Local\Temp\ipykernel_34356\290022593.py:128: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database=db) if db is not None else self.driver.session()


In [ ]:
# Vesualizing the results
# Question 7: Print the route taken by a random bike
random_bike = "W24067" # Select a new random bike from the dataset (change this value)
query = f"""
MATCH (b:Bike {{name: '{random_bike
}'}})<-[:USED]-(t:Trip)-[:STARTED_AT]->(s:Station)
RETURN s.name AS Station, t.startDate AS StartTime
ORDER BY StartTime
"""

# Execute the query and convert the result to a DataFrame
result_df = pd.DataFrame([dict(_) for _ in neo4j_graph.query(query)])
# Print the result
print("Answer to Question 7:")
print(result_df)


In [ ]:
# Close the Neo4j connection
neo4j_graph.close()

# Submission Guidelines

After finishing the assignment save this file and upload it to the D2L. 

Make a 10 mins video of your code and explain each of the answers in the video. Upload the video to YouTube as a private video and provide the link in the following block.

Notes:

* Make sure all the codes inside the requested blocks are clean and runnable. 
* The results of each block should be saved along with queries that resulted them.

In [ ]:
your_video_url = "https://youtu.be/0...."